In [ ]:
# This is necessary to recognize the modules
import os
import sys
import warnings

import pandas as pd

warnings.filterwarnings("ignore")

root_path = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(root_path)

In [ ]:
from geckoterminal_py import GeckoTerminalAsyncClient

gt = GeckoTerminalAsyncClient()

# Criteria
MIN_POOL_AGE_DAYS = 2
MIN_FDV = 70_000
MAX_FDV = 5_000_000
MIN_VOLUME_24H = 150_000
MIN_LIQUIDITY = 50_000
MIN_TRANSACTIONS_24H = 300
NETWORK = "solana"
QUOTE_ASSET = "SOL"


def clean_pools(pools):
    pools["fdv_usd"] = pd.to_numeric(pools["fdv_usd"])
    pools["volume_usd_h24"] = pd.to_numeric(pools["volume_usd_h24"])
    pools["reserve_in_usd"] = pd.to_numeric(pools["reserve_in_usd"])
    pools["pool_created_at"] = pd.to_datetime(pools["pool_created_at"]).dt.tz_localize(None)
    pools["base"] = pools["name"].apply(lambda x: x.split("/")[0].strip())
    pools["quote"] = pools["name"].apply(lambda x: x.split("/")[1].strip())
    pools["volume_liquidity_ratio"] = pools["volume_usd_h24"] / pools["reserve_in_usd"]
    pools["fdv_liquidity_ratio"] = pools["fdv_usd"] / pools["reserve_in_usd"]
    pools["fdv_volume_ratio"] = pools["fdv_usd"] / pools["volume_usd_h24"]
    pools["transactions_h24_buys"] = pd.to_numeric(pools["transactions_h24_buys"])
    pools["transactions_h24_sells"] = pd.to_numeric(pools["transactions_h24_sells"])
    pools["price_change_percentage_h1"] = pd.to_numeric(pools["price_change_percentage_h1"])
    pools["price_change_percentage_h24"] = pd.to_numeric(pools["price_change_percentage_h24"])
    return pools


def filter_pools(
    pools,
    min_pool_age_days=MIN_POOL_AGE_DAYS,
    min_fdv=MIN_FDV,
    max_fdv=MAX_FDV,
    min_volume_24h=MIN_VOLUME_24H,
    min_liquidity=MIN_LIQUIDITY,
    min_transactions_24h=MIN_TRANSACTIONS_24H,
):
    return pools[
        (pools["pool_created_at"] > pd.Timestamp.now() - pd.Timedelta(days=min_pool_age_days))
        & (pools["fdv_usd"] >= min_fdv)
        & (pools["fdv_usd"] <= max_fdv)
        & (pools["volume_usd_h24"] >= min_volume_24h)
        & (pools["reserve_in_usd"] >= min_liquidity)
        & (pools["transactions_h24_buys"] >= min_transactions_24h)
        & (pools["transactions_h24_sells"] >= min_transactions_24h)
    ]


top_pools = await gt.get_top_pools_by_network(NETWORK)
new_pools = await gt.get_new_pools_by_network(NETWORK)
top_pools = filter_pools(clean_pools(top_pools))
new_pools = filter_pools(clean_pools(new_pools))

In [ ]:
top_pools.columns

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler


def cluster_and_graph(pools, n_clusters=5):
    # Prepare data for clustering
    features = [
        "reserve_in_usd",
        "volume_usd_h24",
        "fdv_usd",
        "transactions_h24_buys",
        "transactions_h24_sells",
        "transactions_h1_buys",
        "transactions_h1_sells",
    ]
    X = pools[features].copy()
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)

    # Perform clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    pools["cluster"] = kmeans.fit_predict(X_scaled)
    pools["cluster"] = "Cluster " + pools["cluster"].astype(str)

    # Create 2x2 subplot figure for scatter plots
    fig = make_subplots(
        rows=2,
        cols=2,
        subplot_titles=("Volume vs Liquidity", "FDV vs Liquidity", "FDV vs Volume", "Volume/Liquidity vs FDV/Liquidity"),
    )

    # 1. Volume vs Liquidity scatter plot with clusters
    for cluster in sorted(pools["cluster"].unique()):
        cluster_data = pools[pools["cluster"] == cluster]
        fig.add_trace(
            go.Scatter(
                x=cluster_data["reserve_in_usd"],
                y=cluster_data["volume_usd_h24"],
                mode="markers",
                name=cluster,
                marker=dict(
                    size=cluster_data["transactions_h24_buys"],
                    sizemode="area",
                    sizeref=2.0 * max(pools["transactions_h24_buys"]) / (40.0**2),
                    sizemin=4,
                ),
                hovertemplate=(
                    "Name: %{customdata[0]}<br>"
                    + "Volume/Liquidity: %{customdata[1]:.2f}<br>"
                    + "Buy Transactions: %{customdata[2]}<br>"
                    + "Sell Transactions: %{customdata[3]}<br>"
                    + "FDV: $%{customdata[4]:,.0f}<br>"
                    + "<extra></extra>"
                ),
                customdata=cluster_data[
                    ["name", "volume_liquidity_ratio", "transactions_h24_buys", "transactions_h24_sells", "fdv_usd"]
                ],
            ),
            row=1,
            col=1,
        )

    # 2. FDV vs Liquidity scatter plot
    for cluster in sorted(pools["cluster"].unique()):
        cluster_data = pools[pools["cluster"] == cluster]
        fig.add_trace(
            go.Scatter(
                x=cluster_data["reserve_in_usd"],
                y=cluster_data["fdv_usd"],
                mode="markers",
                name=cluster,
                marker=dict(
                    size=cluster_data["transactions_h24_buys"],
                    sizemode="area",
                    sizeref=2.0 * max(pools["transactions_h24_buys"]) / (40.0**2),
                    sizemin=4,
                ),
                showlegend=False,
                hovertemplate=(
                    "Name: %{customdata[0]}<br>"
                    + "FDV/Liquidity: %{customdata[1]:.2f}<br>"
                    + "Buy Transactions: %{customdata[2]}<br>"
                    + "Sell Transactions: %{customdata[3]}<br>"
                    + "<extra></extra>"
                ),
                customdata=cluster_data[["name", "fdv_liquidity_ratio", "transactions_h24_buys", "transactions_h24_sells"]],
            ),
            row=1,
            col=2,
        )

    # 3. FDV vs Volume scatter plot
    for cluster in sorted(pools["cluster"].unique()):
        cluster_data = pools[pools["cluster"] == cluster]
        fig.add_trace(
            go.Scatter(
                x=cluster_data["volume_usd_h24"],
                y=cluster_data["fdv_usd"],
                mode="markers",
                name=cluster,
                marker=dict(
                    size=cluster_data["transactions_h24_buys"],
                    sizemode="area",
                    sizeref=2.0 * max(pools["transactions_h24_buys"]) / (40.0**2),
                    sizemin=4,
                ),
                showlegend=False,
                hovertemplate=(
                    "Name: %{customdata[0]}<br>"
                    + "FDV/Volume: %{customdata[1]:.2f}<br>"
                    + "Buy Transactions: %{customdata[2]}<br>"
                    + "Sell Transactions: %{customdata[3]}<br>"
                    + "<extra></extra>"
                ),
                customdata=cluster_data[["name", "fdv_volume_ratio", "transactions_h24_buys", "transactions_h24_sells"]],
            ),
            row=2,
            col=1,
        )

    # 4. Volume/Liquidity vs FDV/Liquidity scatter plot
    for cluster in sorted(pools["cluster"].unique()):
        cluster_data = pools[pools["cluster"] == cluster]
        fig.add_trace(
            go.Scatter(
                x=cluster_data["volume_liquidity_ratio"],
                y=cluster_data["fdv_liquidity_ratio"],
                mode="markers",
                name=cluster,
                marker=dict(size=10),
                showlegend=False,
                hovertemplate=(
                    "Name: %{customdata[0]}<br>"
                    + "Volume/Liquidity: %{x:.2f}<br>"
                    + "FDV/Liquidity: %{y:.2f}<br>"
                    + "<extra></extra>"
                ),
                customdata=cluster_data[["name"]],
            ),
            row=2,
            col=2,
        )

    # Update layout for all subplots
    fig.update_xaxes(type="log", row=1, col=1, title_text="Liquidity (USD)")
    fig.update_xaxes(type="log", row=1, col=2, title_text="Liquidity (USD)")
    fig.update_xaxes(type="log", row=2, col=1, title_text="Volume (USD)")
    fig.update_xaxes(type="log", row=2, col=2, title_text="Volume/Liquidity Ratio")

    fig.update_yaxes(type="log", row=1, col=1, title_text="Volume (USD)")
    fig.update_yaxes(type="log", row=1, col=2, title_text="FDV (USD)")
    fig.update_yaxes(type="log", row=2, col=1, title_text="FDV (USD)")
    fig.update_yaxes(type="log", row=2, col=2, title_text="FDV/Liquidity Ratio")

    fig.update_layout(
        height=1000, width=1200, title_text="Key Metrics Analysis by Cluster (size = transactions)", showlegend=True
    )
    fig.show()

    # Price action plots
    fig3 = make_subplots(rows=2, cols=1, subplot_titles=("Price Changes by Cluster", "Transactions by Cluster"))

    for cluster in sorted(pools["cluster"].unique()):
        cluster_data = pools[pools["cluster"] == cluster]

        # Price changes
        fig3.add_trace(
            go.Box(y=cluster_data["price_change_percentage_h24"], name=f"{cluster} 24h", showlegend=True), row=1, col=1
        )
        fig3.add_trace(go.Box(y=cluster_data["price_change_percentage_h1"], name=f"{cluster} 1h", showlegend=True), row=1, col=1)

        # Transactions
        fig3.add_trace(
            go.Box(
                y=cluster_data["transactions_h24_buys"] + cluster_data["transactions_h24_sells"], name=cluster, showlegend=False
            ),
            row=2,
            col=1,
        )

    fig3.update_layout(height=800, title_text="Price Action and Trading Activity by Cluster", boxmode="group")
    fig3.show()

    # Key ratios comparison
    fig4 = make_subplots(rows=1, cols=3, subplot_titles=("Volume/Liquidity", "FDV/Liquidity", "FDV/Volume"))

    for cluster in sorted(pools["cluster"].unique()):
        cluster_data = pools[pools["cluster"] == cluster]

        fig4.add_trace(go.Box(y=cluster_data["volume_liquidity_ratio"], name=cluster), row=1, col=1)
        fig4.add_trace(go.Box(y=cluster_data["fdv_liquidity_ratio"], name=cluster), row=1, col=2)
        fig4.add_trace(go.Box(y=cluster_data["fdv_volume_ratio"], name=cluster), row=1, col=3)

    fig4.update_layout(
        title="Distribution of Key Ratios by Cluster",
        yaxis_type="log",
        yaxis2_type="log",
        yaxis3_type="log",
        height=500,
        showlegend=True,
    )
    fig4.show()

    # Create a summary DataFrame with key metrics by cluster
    summary_stats = (
        pools.groupby("cluster")
        .agg(
            {
                "volume_liquidity_ratio": ["mean", "median", "std"],
                "fdv_liquidity_ratio": ["mean", "median", "std"],
                "fdv_volume_ratio": ["mean", "median", "std"],
            }
        )
        .round(2)
    )

    print("\nSummary Statistics by Cluster:")
    print(summary_stats)

    # Print cluster statistics
    print("\nCluster Size and Averages:")
    cluster_stats = (
        pools.groupby("cluster")
        .agg({"reserve_in_usd": ["count", "mean"], "volume_usd_h24": "mean", "fdv_usd": "mean", "transactions_h24_buys": "mean"})
        .round(2)
    )
    print(cluster_stats)

    return pools, summary_stats

In [ ]:
filtered_top_pools, summary_stats = cluster_and_graph(top_pools, n_clusters=3)
# filtered_new_pools, summary_stats = cluster_and_graph(new_pools, n_clusters=3)

In [ ]:
new_pools[new_pools["address"] == "AJn45NMCTVQ2DQbrjUcY5is21KACGJi2G6FcVYMayAwF"]

In [ ]:
class GeckoT(GeckoTerminalAsyncClient):
    async def get_multiple_tokens_on_network(self, network, token_ids):
        GET_SPECIFIC_TOKEN_ON_NETWORK_PATH = "/networks/{network_id}/tokens/multi/{addresses}"
        response = await self.api_request(
            "GET", GET_SPECIFIC_TOKEN_ON_NETWORK_PATH.format(network_id=network, addresses=token_ids)
        )
        return response["data"]

In [ ]:
gt = GeckoT()

await gt.get_multiple_tokens_on_network(NETWORK, "AJn45NMCTVQ2DQbrjUcY5is21KACGJi2G6FcVYMayAwF")

In [ ]:
top_pools.columns
gt2 = GeckoT()

chunk_size = 30

tokens_info = []
for i in range(0, len(filtered_top_pools), chunk_size):
    token_ids = ",".join(filtered_top_pools["base_token_id"].tolist()[i : i + chunk_size])
    tokens = await gt2.get_multiple_tokens_on_network(NETWORK, token_ids)
    tokens_info.extend(tokens)
    print(len(tokens))

df = pd.DataFrame(tokens_info)

In [ ]:
import json

# Extract relevant information and format it according to the desired structure
formatted_tokens = []
for _, row in df.iterrows():
    attrs = row["attributes"]
    token_info = {
        "chainId": 101,
        "name": attrs["name"],
        "symbol": attrs["symbol"],
        "address": attrs["address"],
        "decimals": attrs["decimals"],
        "logoURI": attrs["image_url"],
        "tags": [],
        "verified": True,
        "holders": None,
    }
    formatted_tokens.append(token_info)
token_list = {
    "name": "Solana Token List",
    "logoURI": "",
    "keywords": ["solana", "spl"],
    "tags": {"lp-token": {"name": "lp-token", "description": ""}, "jupiter": {"name": "jupiter", "description": ""}},
    "timestamp": "2024-07-25T12:10:34.121Z",
    "tokens": formatted_tokens,
}
token_list["tokens"].append(
    {
        "chainId": 101,
        "name": "SOL",
        "symbol": "SOL",
        "address": "11111111111111111111111111111111",
        "decimals": 9,
        "logoURI": "https://coin-images.coingecko.com/coins/images/21629/large/solana.jpg?1696520989",
        "tags": [],
        "verified": True,
        "holders": None,
    },
)
# Save to JSON file
with open("solflare-tokenlist.json", "w") as f:
    json.dump(token_list, f, indent=2)

In [ ]:
pd.DataFrame(token_list["tokens"])

In [ ]:
# Generate YAML config for M3M3D3X strategy
config = {
    "quote_asset": "SOL",
    "quote_asset_allocation": "0.4",
    "min_order_amount": "0.005",
    "max_swaps_per_cycle": 1,
    "sell_deviation_threshold": "0.05",
    "buy_deviation_threshold": "0.20",
    "chunk_size": 30,
    "tradable_tokens": [
        {"name": token["name"], "symbol": token["symbol"], "address": token["address"]} for token in token_list["tokens"]
    ],
}

# Write YAML file
import yaml

with open("m3m3d3x-config.yml", "w") as f:
    yaml.dump(config, f, default_flow_style=False)

In [ ]:
pool = filtered_top_pools.iloc[0]
pool

In [ ]:
ohlcv = await gt.get_ohlcv(NETWORK, pool_address=pool["address"], timeframe="1m")

In [ ]:
from core.data_structures.candles import Candles

ohlcv.index = pd.to_datetime(ohlcv.timestamp, unit="s")

candles = Candles(ohlcv, connector_name="geckoterminal", trading_pair=f"{pool['base']}/{pool['quote']}", interval="1m")
candles.plot()

In [ ]:
len(ohlcv)

In [ ]:
top_pools